# files

Some file handling, searching, recognizing tools.


## visualizing files



### create a word cloud of imports



#### the code

project word-cloud?



In [ ]:
var importer = require('../Core');
var path = require('path');
    
function projectWordCloud(project) {
    var d3CloudToSVG = importer.import('d3.ipynb[create word-cloud]');
    var relativePaths = importer.import('relative paths and includes', {project});
    
    var words = [];

    function wordCount(r) {
        var words = r['packages'].map(p => p.import)
            .concat(r['packages'].map(p => path.basename(p.file)))
            .concat(r['relatives'].map(r => path.basename(r.import)))
            .concat(r['relatives'].map(r => path.basename(r.file)));
        var wordCount = {};
        words.forEach(w => {
            if (typeof wordCount[w] === 'undefined') {
                wordCount[w] = 15;
            } else {
                wordCount[w]++;
            }
        });
        return Object.keys(wordCount).map((d) => ({text: d, size: wordCount[d]}));
    };

    return relativePaths(project)
        .then(words => d3CloudToSVG(wordCount(words)));
}

module.exports = projectWordCloud;

if(typeof $$ !== 'undefined') {
    var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
    var project = PROFILE_PATH + '/Documents/portal';

    $$.async()
    projectWordCloud(project)
        .then(svg => $$.svg(svg))
        .catch(e => $$.sendError(e));
}


### create a tree out of imports


#### the code

project imports d3 tree?



In [ ]:
var importer = require('../Core');
var path = require('path');

function icons(c) {
    return c.replace('.component', '\u2699')
           .replace('.module', '\u26E9')
           .replace('.service', '\u26F4')
           .replace('.routing', '\u2697');
}

function wordCount(r) {
    var words = r['packages'].map(p => p.import)
        .concat(r['packages'].map(p => path.basename(p.file)))
        .concat(r['relatives'].map(r => path.basename(r.import)))
        .concat(r['relatives'].map(r => path.basename(r.file)));
    var wordCounts = {};
    words.forEach(w => {
        if (typeof wordCounts[w] === 'undefined') {
            wordCounts[w] = 1;
        } else {
            wordCounts[w]++;
        }
    });
    var resultWords = Object.keys(wordCounts).map((d, i) => ({
        name: icons(d),
        branch: ((/\.component|\.module|\.service|\.routing/ig).exec(d) || {})[0],
        size: wordCounts[d]}));
    var edges = [];
    r['packages'].forEach(p => edges[edges.length] = {
        source: icons(p.import),
        target: icons(path.basename(p.file))
    });
    r['relatives'].forEach(p => edges[edges.length] = {
        source: icons(path.basename(p.file)),
        target: icons(path.basename(p.import))
    });
    return {nodes: resultWords, edges: edges};
};

var formatNodes;
function projectTree(project) {
    var formatNodes = importer.import('d3.ipynb[format tree]');
    var d3TieredPieSVG = importer.import('d3.ipynb[display d3 tree]');
    var projectRelatives = importer.import('relative paths and includes', {project});
    var words = [];
    
    return projectRelatives(project)
        .then(words => {
            var tree = wordCount(words);
            var nodeNames = tree.nodes.map(n => n.name);
            tree.nodes = tree.nodes
                .filter((n, i, arr) => nodeNames.indexOf(n.name) === i)
                .map((n, i) => {
                    Object.assign(n, {index: tree.nodes
                                      .filter(f => f.branch === n.branch).indexOf(n)})
                    return n;
                })
            var edgeNames = tree.edges.map(n => n.source + '/' + n.target);
            tree.edges = tree.edges
                .filter((n, i, arr) => edgeNames.indexOf(n.source + '/' + n.target) === i)
            var root = formatNodes(tree.nodes, tree.edges);
            return d3TieredPieSVG(root);
        })
        .catch(e => console.log(e))
};
module.exports = projectTree;

if(typeof $$ !== 'undefined') {
    var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
    var project = PROFILE_PATH + '/jupytangular2';

    $$.async()
    projectTree(project)
        .then(svg => $$.svg(svg))
        .catch(e => $$.sendError(e));
}


## listing files

Finding files is easy with glob and minimatch.



### list project files?

List files in a project excluding some excessive searches locations?

TODO: sortLev on all files on no result and find with similar names < 20% difference


#### the code



In [ ]:
var glob = require('glob');
var path = require('path');
var {ignores} = importer.import('common ignore paths');

// TODO: pull all from gitignore library and add to ignore list
//  https://github.com/github/gitignore
// TODO: submit script to github project for fame and fortune!

// But we also want to automatically load projects?
function listInProject(root, match = '{,*,*/,*/*/*,*/*/*/*,*/*/*/*/*}package.json') {
    return glob.sync(match, {
        ignore: ignores,
        cwd: path.resolve(root),
        silent: true,
        nodir: true,
        strict: false
    })
    .map(m => path.join(root, m))
}

module.exports = {
    listInProject
}


### ignoring files

common ignore paths?

TODO: use the one from github, might need to be reformatted for minimatch

https://github.com/github/gitignore


#### the code

In [ ]:
var ignores = [
    '**/node_modules/**',
    '**/packages/**',
    '**/vendor/**',
//            '**/*test*/*',
//            '**/*Test*/*',
    '**/Downloads/**',
    '**/bin/**',
    '**/obj/**',
    '**/Library/**',
    '**/Applications/**',
    '**/AppData/**',
    '**/Trash/**',
    '**/proc/**',
    '**/Music/**',
    '**/wp-content/**',
    '**/wp-includes/**',
    '**/Pods/**',
    '**/svn/**',
    '**/lib/**',
    '**/.git/**',
    '**/.vscode/**',
    '**/.npm/**',
    '**/\\.*',
    '**/.*',
    '**/Cache/**',
    '**/Creative Cloud Files/**',
]

module.exports = {
    ignores
}


### listing projects

Detect projects across multiple frameworks.  List them by name for easy lookup.

TODO: get names out of project files, instead of just parent directory.


#### the code

List all projects by names?



In [ ]:
var importer = require('../Core');
var path = require('path');
var listInProject = importer.import('list project files');

// But we also want to automatically load projects?
function listProjects(root, match = '{,*,*/,*/*/*,*/*/*/*,*/*/*/*/*}+(package.json|NuGet.config|*.sln|*.csproj)') {
    var result = {};
    var matches = listInProject(root, match);
    matches.forEach(m => {
        var projectPath = path.resolve(path.dirname(m));
        result[path.basename(projectPath)] = projectPath;
    });
    return result;
};
module.exports = listProjects;



#### test project files list


In [ ]:
var importer = require('../Core');
var {
    listInProject,
    listProjects
} = importer.import(['list project files', 'list all projects by name']);

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;

if(typeof describe !== 'undefined') {

    describe('file system service', () => {
        beforeEach(() => {
        })

        it('should list files quickly', () => {
            var files = listInProject(PROFILE_PATH);
            console.log(files.length);
            assert(files.length > 0);
        })

        it('should ignore hidden directories for better performance', () => {
            var files = listProjects(PROFILE_PATH);
            console.log(files.portal);
            assert(files.jupytangular && files.jupytangular.indexOf('jupytangular') > -1
                    || files.jupytangular2 && files.jupytangular2.indexOf('jupytangular2') > -1, 'could not find own notebook project under the name jupytangular');
        })
    });

}


## fixing files



### find similar filename

Useful when a require or import path changes, this can automatically find files that moved but kept the same name.




#### the code

Find similar filename in project?


In [ ]:
var glob = require('glob');

function findSimilarFile(base, project) {
    return new Promise((resolve, reject) => {
        var searchStrings = '**/' + base + '*';
        console.log('Search for ' + searchStrings);
        glob(searchStrings, {
            ignore: ['**/node_modules/**'],
            cwd: project
        }, function (err, matches) {
            var isEmpty = matches.length == 0;
            if (err) {
                return reject(err);
            } else if (isEmpty) {
                console.log('No matches for ' + searchStrings)
                return resolve();
            }

            // TODO: try to build and make sure it works before patching
            resolve(matches[0].replace('.ts', ''));
        });
    });
};

module.exports = findSimilarFile;



### fix all imports in a project


#### the code

fix project paths?

TODO: make this syntax aware using transpiler

TODO: find file based on matching syntax structure

TODO: convert this to a fix for eslint or transpiler


In [ ]:
var fs = require('fs')
var path = require('path')
var minimatch = require('minimatch');
var importer = require('../Core')
var {relativeImports} = importer.import('builtin and local modules')

function fixImports(project) {
    var packagePath = path.join(project, 'package.json')
    var codeFiles = listInProject(project, '**/*.+(ts|js|ipynb)')
    codeFiles.forEach(f => {
        var imports = relativeImports(f)
        
        // TODO: find file based on matching syntax structure
        imports.missing.forEach(i => {
            var match = codeFiles
                .filter(f => path.basename(f) === path.basename(i))[0]
            if(match) {
                var newRelative = './' + path.relative(
                    path.dirname(f), path.join(project, match))
                var fixFile = fs.readFileSync(f).toString()
                fs.writeFileSync(f, fixFile.replace(i, newRelative))
            }
        })
        
        // fix package json
        if(fs.existsSync(packagePath)) {
            var packageJson = JSON.parse(fs.readFileSync(packagePath))
            var toAdd = imports.packages
                .filter(p => typeof packageJson.dependencies[p] === 'undefined'
                        && typeof packageJson.devDependencies[p] === 'undefined')
            toAdd.forEach(a => {
                console.log(`adding ${a}`)
                packageJson.dependencies[a] = 'latest'
            })
            if(toAdd.length > 0) {
                fs.writeFileSync(packagePath,
                                 JSON.stringify(packageJson, null, 4))
            }
        }
    })
}

module.exports = {
    fixImports
}


#### TODO: test fix imports

Using memory-fs make a code file, move a reference, check the file was corrected.


### find functions that are too long

(?=\{((?:[^{}]++|\{(?1)\})++)\})

TODO: turn this in to a bs-lint test

TODO: use a recusive decent parser to apply to all languages, test finding long python functions which are white-space delimited


#### the code

Find too long of functions using regexp?



In [ ]:
var importer = require('../Core');
var gulp = require('gulp');
var tap = require('gulp-tap');

function matchCurlyBraces(contents) {
    var XRegExp = require('xregexp');
    var newMatches = XRegExp.matchRecursive(contents, '{', '}', 'gi', {
        valueNames: ['literal', null, 'value', null]
    }).filter(m => m.name === 'value');
    //console.log(newMatches);
    return newMatches.reduce(
        (arr, m) => arr.concat(matchCurlyBraces(m.value)),
        newMatches);
};

function findLongFunctions(project) {
    var tasksToPromise = importer.import('tasks to promise', {gulp});

    var results = [];
    gulp.task('find long functions', function () {
        return gulp.src('**/*.+(ts|js|cs)', {
            ignore: ['**/node_modules/**',
                '**/dist/**',
                '**/typings/**',
                '**/packages/**',
                '**/*.spec*',
                '**/*.Tests/**'
            ], cwd: project
        })
            .pipe(tap(file => {
                const contents = file.contents.toString();
                try {
                    results = results.concat(matchCurlyBraces(contents)
                        .map(m => Object.assign(m, {path: file.path})));
                } catch (e) {
                    results = results.concat([{
                        name: 'value',
                        value: file.contents.toString(),
                        start: 0,
                        end: file.contents.length,
                        error: e,
                        path: file.path
                    }]);
                }
            }));
    });

    return tasksToPromise(['find long functions'])
        .then(() => {
            var code = results.map(r => r.value);
            var topLevel = results
                .filter(r => code
                    .filter(c => c !== r.value && c.indexOf(r.value) > 0).length === 0);
            results = results
                .filter(m => topLevel.filter(c => c.value === m.value).length === 0);
            code = results.map(r => r.value);
            topLevel = topLevel.concat(results.filter(r => code
                .filter(c => c !== r.value && c.indexOf(r.value) > 0).length === 0));
            results = results
                .filter(m => topLevel.filter(c => c.value === m.value).length === 0);
            return results
                .filter(m => m.value.split(/\n/ig).length > 100)
        });
};

module.exports = findLongFunctions;


#### test find too long of functions


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/asm';

if(typeof $$ !== 'undefined') {
    $$.async();
    findLongFunctions(project)
        .then(e => $$.sendResult(e.map(m => m.path)))
        .catch(e => $$.sendError(e))

}


## file tools

### match filename?

Minimatching from an object full of file names and their output destination.


In [ ]:
var minimatch = require('minimatch');

var getDirectory = (match) => match[match.length-1] === '/'
    ? match
    : (fs.existsSync(match) && fs.lstatSync(match).isDirectory()
       ? match
       : path.dirname(match));


function matchFilename(filename, matchOutput, projectOutput) {
    return Object.keys(matchOutput)
    .filter(match => minimatch(filename, match))
    .map(match => {
        var projectMatch = path.join(projectOutput, matchOutput[match])
        var dir = getDirectory(projectMatch);
        return path.join(dir, path.basename(projectMatch) === path.basename(dir)
            ? filename
            : path.basename(matchOutput[match]));
    })
}

module.exports = {
    matchFilename,
    getDirectory
}


### make directories recursively

mkdirp?


In [ ]:
var fs = require('fs');
var path = require('path');

function mkdirpSync(p) {
    if(!fs.existsSync(p)) {
        try {
            fs.mkdirSync(p);
        } catch (e) {
            if(e.code === 'ENOENT') {
                mkdirpSync(path.dirname(p));
                fs.mkdirSync(p);
            }
        }
    }
}

module.exports = mkdirpSync;


### mocking file-system calls

This can be useful if a library doesn't natively support streams or putting content in to memory instead of directly on disk.



#### the code

How to use memory-fs and rewire to audit cli events?



In [ ]:
var path = require('path');

// TODO: use https://github.com/jhnns/rewire for replacing variables, something like https://github.com/mariocasciaro/object-path?

function mockTypescriptFs(root, data) {
    var mock = require('mock-require');
    var memfs = require('memfs');
    var mem = new memfs.Volume;
    var unionfs = require('unionfs');
    var fs = require('fs');

    mem.mountSync(root, data);

    // Create a union of two file systems:
    unionfs
        .use(fs)
        .use(mem)
        .replace(fs);

    var ts = require('typescript');
    var tsMock = Object.assign({}, ts);
    var toMock = {
        readFile: 'readFileSync',
        realpath: 'realpathSync',
        writeFile: 'writeFileSync',
        fileExists: 'existsSync',
        directoryExists: 'existsSync',
        createDirectory: 'mkdirSync',
        getDirectories: 'readdirSync'
    };
    Object.keys(toMock)
        .forEach(k => {
            if (typeof mem[toMock[k]] === 'function') {
                tsMock.sys[k] = function () {
                    // typescript.sys functions return defined on error
                    try {
                        return mem[toMock[k]].apply(mem, arguments);
                    } catch (e) {
                        return;
                    }
                };
            }
        });

    mock('typescript', tsMock);
    return mock;
};
module.exports = mockTypescriptFs;

// TODO: search github for projects like graceful-fs and then search google for projects that use graceful-fs and see if we can record some filesystem activity



## TODO: use static code analysis to find consecutive words in template files?

https://stackoverflow.com/questions/18717536/in-python-how-can-i-distinguish-between-a-human-readible-word-and-random-string

TODO: combine with DOM minimizing service from recording app?

TODO: use to find the main content of the page for scraping.


In [ ]:
var fs = require('fs');
var glob = require('glob');
fs.readdirSync('.');
var PROJECT_PATH = '/Users/briancullinan/Google Drive/Colab Notebooks'
glob.sync('**/*.ipynb', {cwd: PROJECT_PATH}).forEach(f => {
    const nb = JSON.parse(fs.readFileSync(path.join(PROJECT_PATH, f)).toString());
    nb.nbformat_minor = 0;
    /*
    nb.metadata = {
        colab: {
          "name": path.basename(f),
          "version": "0.3.2",
          "views": {},
          "default_view": {},
          "provenance": []
        },
        kernelspec: {
          "name": "javascript",
          "display_name": "Javascript"
        }
    };
    */
    fs.writeFileSync(path.join(PROJECT_PATH, f), JSON.stringify(nb, null, 2));
});


## TODO:

TODO: make a utulity for displaying files that match specific code.

TODO: make a utility for loading file information like PECL ID3 from PHP.
